In [ ]:
# Install OpenAI's Whisper library for automatic speech recognition (ASR)
# This allows us to transcribe audio files into text.
!pip install -q openai-whisper

# Install FFmpeg, a multimedia processing tool for handling audio and video files.
# FFmpeg helps in converting, compressing, and preparing audio files for transcription.
!pip install -q ffmpeg

# Install Pydub, a Python library for audio processing.
# Pydub is used to manipulate audio files (e.g., cutting, merging, or changing formats).
!pip install -q pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 14.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.5 MB/s eta 0:00:00


In [ ]:
import whisper
model = whisper.load_model("medium", device="cuda")

100%|█████████████████████████████████████| 1.42G/1.42G [00:27<00:00, 54.8MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [ ]:
# Import necessary libraries for handling audio, base64 encoding, and interaction with JavaScript in a Colab notebook
from IPython.display import Javascript  # To display JavaScript code in a notebook cell
from google.colab import output  # To run JavaScript code in a Google Colab environment
from base64 import b64decode  # To decode base64-encoded audio data
from io import BytesIO  # To treat the audio data as a file-like object
from pydub import AudioSegment  # To manipulate and handle audio data (e.g., converting formats)

# JavaScript code for recording audio in the browser
JAVASCRIPT_RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))  // Helper function to pause for 'time' milliseconds
const b2text = blob => new Promise(resolve => {  // Convert a Blob (audio file) to a base64 text format
  const reader = new FileReader()  // Initialize FileReader to read the audio file
  reader.onloadend = e => resolve(e.srcElement.result)  // Resolve the base64 data once read
  reader.readAsDataURL(blob)  // Start reading the Blob data as a base64 data URL
})
var record = time => new Promise(async resolve => {  // Main recording function
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })  // Request access to the user's microphone
  recorder = new MediaRecorder(stream)  // Create a new MediaRecorder instance to capture audio
  chunks = []  // Array to store audio data chunks
  recorder.ondataavailable = e => chunks.push(e.data)  // Collect audio data chunks as they are available
  recorder.start()  // Start recording
  await sleep(time)  // Wait for the specified duration (in milliseconds)
  recorder.onstop = async ()=>{  // Once recording stops, process the data
    blob = new Blob(chunks)  // Combine the chunks into a single Blob
    text = await b2text(blob)  // Convert the Blob into a base64 text string
    resolve(text)  // Return the base64 string to the caller
  }
  recorder.stop()  // Stop recording after the specified time
})
"""

# Function to record audio and return it as an AudioSegment object
def record(sec=3):  # Default record duration is 3 seconds
  print("Start recording...")  # Notify the user that recording has started
  display(Javascript(JAVASCRIPT_RECORD))  # Run the JavaScript code to record audio in the browser
  s = output.eval_js('record(%d)' % (sec*1000))  # Call the JavaScript record function (convert seconds to milliseconds)
  b = b64decode(s.split(',')[1])  # Decode the base64-encoded audio data
  audio = AudioSegment.from_file(BytesIO(b))  # Convert the audio data into an AudioSegment object for further manipulation
  return audio  # Return the AudioSegment object containing the recorded audio

# Function to record audio and transcribe it using a speech-to-text model
def stt(sec=5):  # Default record duration is 5 seconds
  print("Start recording...")  # Notify the user that recording has started
  display(Javascript(JAVASCRIPT_RECORD))  # Run the JavaScript code to record audio in the browser
  s = output.eval_js('record(%d)' % (sec*1000))  # Call the JavaScript record function (convert seconds to milliseconds)
  b = b64decode(s.split(',')[1])  # Decode the base64-encoded audio data
  with open('/content/audio.wav', "wb") as f:  # Write the decoded audio to a .wav file
    f.write(b)  # Save the audio to a file
  print("Finish recording. Send to model")  # Notify that the recording is finished and will be sent to the model
  result = model.transcribe("/content/audio.wav", language="vietnamese")  # Transcribe the audio using the model (assumes a model is loaded)
  return result["text"]  # Return the transcribed text

In [ ]:
# Call the function Speech-To-Text using Open-AI Whisper model
stt()

Start recording...


<IPython.core.display.Javascript object>

Finish recording. Send to model


' Sinh viên kỳ 4 FPT Chiên ngành Chí thể Nhân tạo'

In [ ]:
# Install the 'transformers' library from Hugging Face
# This library provides pre-trained models for natural language processing (NLP) tasks such as text generation, translation, summarization, and more.
!pip install -q transformers

# Install 'pydub' library for audio processing
# Pydub is used to manipulate audio files, such as cutting, merging, converting formats, and applying effects.
# It simplifies tasks like working with audio data in Python.
!pip install -q pydub

In [ ]:
# Import the 'pipeline' function from the Hugging Face 'transformers' library.
from transformers import pipeline

# Initialize the Automatic Speech Recognition (ASR) pipeline with the pre-trained 'PhoWhisper-large' model.
# This model is trained specifically for Vietnamese speech recognition and can transcribe audio into text.
transcriber = pipeline("automatic-speech-recognition", model="vinai/PhoWhisper-large", device="cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/4.26k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

In [ ]:
# Import necessary libraries for handling audio, base64 encoding, and interaction with JavaScript in a Colab notebook
from IPython.display import Javascript  # To display and run JavaScript code in a notebook cell
from google.colab import output  # To run JavaScript code in a Google Colab environment
from base64 import b64decode  # To decode base64-encoded audio data
from io import BytesIO  # To treat the audio data as a file-like object for processing
from pydub import AudioSegment  # To manipulate and handle audio data (e.g., converting formats)

# JavaScript code for recording audio in the browser using the user's microphone
JAVASCRIPT_RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))  // Helper function to pause for 'time' milliseconds
const b2text = blob => new Promise(resolve => {  // Convert a Blob (audio file) to a base64 text format
  const reader = new FileReader()  // Initialize FileReader to read the audio file
  reader.onloadend = e => resolve(e.srcElement.result)  // Resolve the base64 data once read
  reader.readAsDataURL(blob)  // Start reading the Blob data as a base64 data URL
})
var record = time => new Promise(async resolve => {  // Main recording function
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })  // Request access to the user's microphone
  recorder = new MediaRecorder(stream)  // Create a new MediaRecorder instance to capture audio
  chunks = []  // Array to store audio data chunks
  recorder.ondataavailable = e => chunks.push(e.data)  // Collect audio data chunks as they are available
  recorder.start()  // Start recording
  await sleep(time)  // Wait for the specified duration (in milliseconds)
  recorder.onstop = async ()=>{  // Once recording stops, process the data
    blob = new Blob(chunks)  // Combine the chunks into a single Blob
    text = await b2text(blob)  // Convert the Blob into a base64 text string
    resolve(text)  // Return the base64 string to the caller
  }
  recorder.stop()  // Stop recording after the specified time
})
"""

# Function to record audio for a given duration and transcribe it using a pre-trained ASR model
def stt_pho(sec=5):  # Default record duration is 5 seconds
  print("Start recording...")  # Notify the user that recording has started
  display(Javascript(JAVASCRIPT_RECORD))  # Run the JavaScript code to record audio in the browser
  s = output.eval_js('record(%d)' % (sec*1000))  # Call the JavaScript record function (convert seconds to milliseconds)
  b = b64decode(s.split(',')[1])  # Decode the base64-encoded audio data into binary format
  with open('/content/audio.wav', "wb") as f:  # Save the decoded audio to a .wav file
    f.write(b)  # Write the binary data to the file
  print("Finish recording. Send to model")  # Notify that the recording is complete and will be sent to the model
  result = transcriber("/content/audio.wav")  # Use the pre-trained ASR model (e.g., PhoWhisper-large) to transcribe the audio
  return result["text"]  # Return the transcribed text from the model's output


In [ ]:
# Call the function Speech-To-Text using Pho Vietnamese language model
stt_pho()

Start recording...


<IPython.core.display.Javascript object>

Finish recording. Send to model


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


'sinh viên kỳ bốn fpt chuyên ngành trí tuệ nhân tạo.'